In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!pip install chromadb
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install --quiet lark datasets bitsandbytes einops

In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [ ]:
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
from peft import (get_peft_config,
                  get_peft_model,
                  get_peft_model_state_dict,
                  set_peft_model_state_dict,
                  PeftType,
                  PromptEncoderConfig,)
import torch
import numpy as np

padding_side = "right"

if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


def tokenize_function(examples):
    outputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)
    return outputs

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 130,556,800 || all params: 7,052,286,592 || trainable%: 1.851269064250757


In [ ]:
import torch
from transformers import (FalconForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer,
                          DataCollatorWithPadding)

model_name = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_compute_dtype=torch.float16,
)

In [ ]:
model = FalconForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    return_dict=True,
    use_cache=False
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
padding_side = "right"
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                                  padding_side=padding_side)


In [ ]:
!pip install sentence-transformers

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install langchain

In [ ]:
import chromadb
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings

def create_falcon_instruct(model):
        padding_side = "right"
        tokenizer = AutoTokenizer.from_pretrained(model_name,
                                                  padding_side=padding_side)
        hf_pipeline = pipeline(
                task="text-generation",
                model = model,
                tokenizer = tokenizer,
                trust_remote_code = True,
                device_map="auto"
            )
        return hf_pipeline

# def create_sbert_mpnet():
#         EMB_SBERT_MPNET_BASE = "sentence-transformers/all-mpnet-base-v2"
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#         return HuggingFaceEmbeddings(model_name=EMB_SBERT_MPNET_BASE,
#                                      model_kwargs={"device": device})


In [ ]:
vito_pipeline = create_falcon_instruct(model)

In [ ]:
user_question = "What are general recycling principles?"

sequences = vito_pipeline(
		user_question,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: What are general recycling principles?
The general recycling principles are to reduce the amount of waste produced, to reuse and repurpose materials to their fullest potential, and to recycle materials as much as possible.


In [ ]:
import chromadb
from chromadb.config import Settings
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False, allow_reset=True))

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
import json
import os

List_Data = []
List_Articles = os.listdir("Stoffen")

for i in List_Articles:
  try:
    f = open("Stoffen/" + i)
    data = json.load(f)
    List_Data.append(data)
    f.close()
  except:
    print("Something went wrong")

In [ ]:
List_Data[0]

{'artikel': 'Artikel 31 Modellen van gezondheidscertificaten en verklaringen voor de invoer en doorvoer',
 'inhoud': 'Zendingen dierlijke bijproducten en afgeleide producten die bedoeld zijn voor invoer in of doorvoer door de Unie, gaan op het punt van binnenkomst in de Unie waar de veterinaire controles overeenkomstig Richtlijn 97/78/EG plaatsvinden, vergezeld van gezondheidscertificaten en verklaringen volgens de modellen in bijlage XV bij deze verordening.\n\n[Bij wijze van afwijking van de eerste alinea moeten de dierlijke bijproducten en afgeleide producten die afkomstig zijn van en, nadat een derde land de binnenkomst ervan heeft geweigerd, terugkeren naar de Unie, voldoen aan de specifieke eisen in bijlageXIV, hoofdstukVI.]\n',
 'id': '60496',
 'metadata': [{'toplayer': 'Stoffen'}]}

In [ ]:
for i in List_Data:
  List_Data[List_Data.index(i)]["inhoud"] = i["inhoud"].replace("\r", "").replace("\n", "").replace("\t", "")

In [ ]:
from langchain.schema import Document
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
default_ef = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
docs = []
for i in List_Data:
  docs.append(
      Document(
          page_content = i["inhoud"],
          metadata = {"source": i["artikel"], "toplayer": i["metadata"][0]["toplayer"]}
      )
  )

In [ ]:
vectorDB = Chroma.from_documents(docs, default_ef)

In [ ]:
for i in List_Data:
  vectorDB.add_texts(
      texts=[i["inhoud"]],
      metadatas=[{"source": i["artikel"], "toplayer": i["metadata"][0]["toplayer"]}],
      ids=[i["id"]]
  )

In [ ]:
hf_llm = HuggingFacePipeline(pipeline=vito_pipeline)
retriever = vectorDB.as_retriever(search_kwargs={"k":5})
qa = RetrievalQA.from_chain_type(llm=hf_llm, chain_type="stuff",retriever=retriever)

In [ ]:
results = vectorDB.query(
    query_texts=[Input],
    n_results=4
)

In [ ]:
question_template = """
    question: {question}
    context: {context}
    answer:
    """
question_prompt = PromptTemplate(
      template=question_template, input_variables=["context", "question"]
)
qa.combine_documents_chain.llm_chain.prompt = question_prompt

In [ ]:
question = "What document has this name Bijlage 2.4.2. Milieukwaliteitsnormen voor bodem"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
result = qa({"query":question,})

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new StuffDocumentsChain chain...


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 1881, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.


In [ ]:
Input = "Elke eigenaar van een publieke constructie met risicobouwjaar is verplicht om: 1° 	tegen 1 januari 2040 zijn constructie met risicobouwjaar asbestveilig te maken; 2° 	de asbestveilige toestand na 1 januari 2040 te behouden. In het inspectieprotocol, vermeld in artikel 33/10, § 3, kunnen nadere regels worden vastgesteld omtrent de risico-evaluatie en het veilig beheer van asbesthoudende materialen. Voor publieke constructies met risicobouwjaar waarvoor de eigenaar overeenkomstig artikel 33/9 over een asbestinventarisattest moet beschikken, bewijst dit attest of al dan niet aan de verplichting, vermeld in het eerste lid, 1°, is voldaan. Voor de andere publieke constructies met risicobouwjaar kan de Vlaamse Regering bepalen hoe de naleving van deze verplichting kan worden aangetoond."

In [ ]:
results

{'ids': [['77369', '77368', '77373', '77374']],
 'distances': [[0.05653415620326996,
   0.27181005477905273,
   0.3901275396347046,
   0.5912882685661316]],
 'metadatas': [[{'source': 'Artikel  33/6.', 'toplayer': 'Stoffen'},
   {'source': 'Artikel  33/5.', 'toplayer': 'Stoffen'},
   {'source': 'Artikel  33/9.', 'toplayer': 'Stoffen'},
   {'source': 'Artikel 33/10.', 'toplayer': 'Stoffen'}]],
 'embeddings': None,
 'documents': [['Elke eigenaar van een publieke constructie met risicobouwjaar is verplicht om:1tegen 1 januari 2040 zijn constructie met risicobouwjaar asbestveilig te maken;2de asbestveilige toestand na 1 januari 2040 te behouden.[...][...] In het inspectieprotocol, vermeld in artikel 33/10,  3, kunnen nadere regels worden vastgesteld omtrent de risico-evaluatie en het veilig beheer van asbesthoudende materialen.Voor publieke constructies met risicobouwjaar waarvoor de eigenaar overeenkomstig artikel 33/9 over een asbestinventarisattest moet beschikken, bewijst dit attest of